# Aggrégation des fichiers de data.gouv.fr pour les PLF et PLR

In [1]:
import csv
import os

from tqdm import tqdm

## Décodage des fichiers
Les fichiers ne sont pas dans le bon encoding, il faut les réécrire.

In [2]:
RAW_FOLDER = './raw_data/plf_2019'
DECODED_FOLDER = './decoded_data/plf_2019'

if not os.path.exists(DECODED_FOLDER):
    os.makedirs(DECODED_FOLDER)

In [3]:
for fileName in os.listdir(RAW_FOLDER):
    os.system('iconv -f ISO-8859-1 -t UTF-8 {} > {}'.format(os.path.join(RAW_FOLDER, fileName), os.path.join(DECODED_FOLDER, fileName)))

## PLF

Pour une année donnée, on récupère les **Autorisations d'engagements** et les **Crédits de Paiement** par `Type de mission` > `Mission` > `Programme` > `Action`.

Le but est de générer un fichier avec les colonnes suivantes :<br>
`Type de mission` ; `Code Type de Mission` ; `Mission` ; `Code Mission` ; `Programme` ; `Code Programme` ; `Action` ; `Code Action` ; `Autorisations d'engagements` ; `Crédits de paiement`

### 2019

In [4]:
SOURCE_FOLDER = './decoded_data/plf_2019'
TARGET_FOLDER = './parsed_data'

type_de_mission_to_code = {
    "Budget général": "BG",
    "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
    "Comptes de concours financiers": "CF"
}

In [5]:
mission_to_code = {}

with open(os.path.join(SOURCE_FOLDER, 'plf2019-nomenclature-mpa.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[2]
        mission = row[3]

        mission_to_code[mission] = code_mission

In [18]:
type_de_mission_to_filename = {
    "Budget général": "bg",
    "Budgets annexes": "ba",
    "Comptes d'affectation spéciale": "cs",
    "Comptes de concours financiers": "cf"
}

with open(os.path.join(TARGET_FOLDER, 'plf_2019.csv'), 'w+') as output_file:
    # Écrire les headers
    output_file.write("Type de mission;Code type de mission;Mission;Code mission;Programme;Code programme;Action;Code action;Autorisations d'engagement;Crédits de paiement\n")

    # On ouvre chacun des fichiers...
    for type_de_mission in type_de_mission_to_code:
        code_type_de_mission = type_de_mission_to_code[type_de_mission]
        code_filename = type_de_mission_to_filename[type_de_mission]

        with open(os.path.join(
            SOURCE_FOLDER,
            'plf2019-{}-msn-dest.csv'.format(code_filename)), 'r') as csv_file:
            
            csv_reader = csv.reader(csv_file, delimiter=';')
            # On saute la première ligne parce que c'est un header
            next(csv_reader, None)
            
            for index, row in enumerate(csv_reader):
                # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
                if (row[2] == ''):
                    continue
                    
                credits_paiement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
                autorisations_engagement = int(row[8].replace(' ', '')) if row[8].replace(' ', '') != '' else 0

                # On écrit dans le fichier cible toutes les informations utiles
                output_file.write('{};{};{};{};{};{};{};{};{};{}\n'.format(
                    type_de_mission,
                    code_type_de_mission,
                    row[2],
                    mission_to_code[row[2]],
                    row[4],
                    row[3],
                    row[6],
                    row[5],
                    autorisations_engagement,
                    credits_paiement
                ))

ValueError: invalid literal for int() with base 10: "Pensionsmilitairesd'invaliditédevictimesdeguerreetallocationsrattachées"

### Tests génériques

In [11]:
TEST_FILENAME = 'plf_2019.csv'

Vérifie que les lignes ont le bon nombre de colonnes.

In [12]:
with open(os.path.join(TARGET_FOLDER, TEST_FILENAME), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')

    for index, row in enumerate(csv_reader):
        try:
            assert(len(row) == 10)
        except:
            print("Assertion error at line {}".format(index))
            print(*row)

Vérifie qu'il n'y a pas d'action, de CP ou d'AE vides.

In [17]:
with open(os.path.join(TARGET_FOLDER, TEST_FILENAME), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')

    for index, row in enumerate(csv_reader):
        try:
            assert(row[6] != '' and row[8] != '' and row[9] != '')
        except:
            print("Assertion error at line {}".format(index))
            print(row[6], row[8], row[9])

Assertion error at line 68
Actions de co-développement  
Assertion error at line 76
Solidarité  
Assertion error at line 79
Solidarité  
Assertion error at line 99
Grand Paris  
Assertion error at line 105
Plan gouvernemental sur le Marais Poitevin - Poitou Charentes  1594345
Assertion error at line 169
Préparation des forces aériennes  
Assertion error at line 176
Surcoûts liés aux opérations intérieures  
Assertion error at line 235
Dissuasion  76526998
Assertion error at line 241
Dissuasion  148478840
Assertion error at line 244
Commandement et maîtrise de l'information  64611295
Assertion error at line 246
Commandement et maîtrise de l'information  100129950
Assertion error at line 249
Commandement et maîtrise de l'information  189444531
Assertion error at line 251
Commandement et maîtrise de l'information  105078113
Assertion error at line 252
Commandement et maîtrise de l'information  70308678
Assertion error at line 253
Commandement et maîtrise de l'information  56008160
Asserti